## Setup and Initialization

In [45]:
# Import Necessary Libraries
import pandas as pd
import sqlite3 as sql

# Set display options for Pandas DataFrame
pd.set_option('display.max_colwidth', None)

# Connect to the SQLite database
conn = sql.connect('sql-murder-mystery.db')

## Database Exploration

In [46]:
# Query to list all tables in the database
query_tables = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query_tables, conn)
print("Available tables:\n", tables)

Available tables:
                      name
0      crime_scene_report
1         drivers_license
2  facebook_event_checkin
3               interview
4      get_fit_now_member
5    get_fit_now_check_in
6                solution
7                  income
8                  person


In [43]:
# Function to get foreign key information for a given table
def get_foreign_keys(table_name, conn):
    query_fk = f"PRAGMA foreign_key_list({table_name});"
    foreign_keys = pd.read_sql_query(query_fk, conn)
    return foreign_keys

# Dictionary to store foreign key relationships
relationships = {}

# Iterate through all tables and collect foreign key information
for table in tables['name']:
    foreign_keys = get_foreign_keys(table, conn)
    if not foreign_keys.empty:
        relationships[table] = foreign_keys

# Print the connections between tables
print("Connections between tables:")
for table, fks in relationships.items():
    for index, row in fks.iterrows():
        print(f"Table '{table}' has a foreign key '{row['from']}' "
              f"that references '{row['table']}.{row['to']}'")


Connections between tables:
Table 'facebook_event_checkin' has a foreign key 'person_id' that references 'person.id'
Table 'interview' has a foreign key 'person_id' that references 'person.id'
Table 'get_fit_now_member' has a foreign key 'person_id' that references 'person.id'
Table 'get_fit_now_check_in' has a foreign key 'membership_id' that references 'get_fit_now_member.id'
Table 'person' has a foreign key 'license_id' that references 'drivers_license.id'
Table 'person' has a foreign key 'ssn' that references 'income.ssn'


## Data Analysis

In [18]:
# Query to get the crime scene report for the specified date, type, and city
query_crime_scene = """
SELECT *
FROM crime_scene_report
WHERE date = '20180115' AND type='murder' AND city = 'SQL City';
"""
crime_scene_report = pd.read_sql_query(query_crime_scene, conn)
crime_scene_report

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


### First Witness Data

In [31]:
# First witness: Find the last house on "Northwestern Dr"
query_first_witness = """
SELECT *
FROM person
WHERE address_street_name='Northwestern Dr'
ORDER BY address_number DESC
LIMIT 1;
"""
first_witness = pd.read_sql_query(query_first_witness, conn)
first_witness

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [20]:
# Query to get the interview transcript of the first witness
query_first_witness_interview = """
SELECT *
FROM interview 
WHERE person_id = (SELECT id
                   FROM person
                   WHERE address_street_name='Northwestern Dr'
                   ORDER BY address_number DESC
                   LIMIT 1);
"""
first_witness_interview = pd.read_sql_query(query_first_witness_interview, conn)
first_witness_interview

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."


In [21]:
# Get the suspect from the description given by the first witness
query_suspects = """
SELECT *
FROM get_fit_now_member
WHERE id LIKE "48Z%" AND membership_status = "gold";
"""
suspects = pd.read_sql_query(query_suspects, conn)
suspects

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


In [22]:
# Get details of the suspects
query_suspects_details = """
SELECT *
FROM person 
WHERE id IN (SELECT person_id
             FROM get_fit_now_member
             WHERE id LIKE "48Z%" AND membership_status = "gold");
"""
suspects_details = pd.read_sql_query(query_suspects_details, conn)
suspects_details

,id,name,license_id,address_number,address_street_name,ssn
0,28819,Joe Germuska,173289,111,Fisk Rd,138909730
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [23]:
# Get license details of the suspects based on the car plate description
query_license = """
SELECT *
FROM drivers_license
WHERE plate_number LIKE '%H42W%' AND gender = 'male';
"""
license_details = pd.read_sql_query(query_license, conn)
license_details

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [34]:
# Get the person details of the suspects based on their license ID
query_suspects_license = """
SELECT *
FROM person 
WHERE license_id IN (SELECT id
                     FROM drivers_license
                     WHERE plate_number LIKE '%H42W%' AND gender = 'male');
"""
license_suspects = pd.read_sql_query(query_suspects_license, conn)
license_suspects

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [32]:
# Get the suspects from the description given by the first witness
query_suspects_first = """
SELECT *
FROM person
WHERE id IN (
    SELECT person_id
    FROM get_fit_now_member
    WHERE id LIKE '48Z%' AND membership_status = 'gold'
)
AND license_id IN (
    SELECT id
    FROM drivers_license
    WHERE plate_number LIKE '%H42W%' AND gender = 'male'
);
"""
suspects_first = pd.read_sql_query(query_suspects_first, conn)
suspects_first

,id,name,license_id,address_number,address_street_name,ssn
0,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


### Second witness Data

In [35]:
# Second witness: Annabel, who lives on "Franklin Ave"
query_second_witness = """
SELECT *
FROM person
WHERE name LIKE "%Annabel%" AND address_street_name="Franklin Ave";
"""
second_witness = pd.read_sql_query(query_second_witness, conn)
second_witness

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [27]:
# Find the interview transcript of the second witness
query_second_witness_interview = """
SELECT *
FROM interview
WHERE person_id IN (SELECT id
                    FROM person
                    WHERE name LIKE "%Annabel%" AND address_street_name="Franklin Ave");
"""
second_witness_interview = pd.read_sql_query(query_second_witness_interview, conn)
second_witness_interview

,person_id,transcript
0,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [28]:
# Find the check-in details from the gym on the specific date
query_check_in_details = """
SELECT *
FROM get_fit_now_check_in
WHERE membership_id = (SELECT id
                       FROM get_fit_now_member
                       WHERE person_id = (SELECT person_id
                                          FROM interview
                                          WHERE person_id IN (SELECT id
                                                              FROM person
                                                              WHERE name LIKE "%Annabel%"
                                                              AND address_street_name="Franklin Ave")));
"""
check_in_details = pd.read_sql_query(query_check_in_details, conn)
check_in_details

,membership_id,check_in_date,check_in_time,check_out_time
0,90081,20180109,1600,1700


In [29]:
# Query to get all check-ins at the gym on January 9th, 2018 between 4:00 PM and 5:00 PM excluding Annabel
query_suspects_gym = """
SELECT *
FROM get_fit_now_check_in
WHERE check_in_date = '20180109'
  AND (check_in_time BETWEEN '1600' AND '1700'
       OR check_out_time BETWEEN '1600' AND '1700') 
  AND membership_id != 90081;
"""
suspects_gym = pd.read_sql_query(query_suspects_gym, conn)
suspects_gym

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


In [30]:
# Query to get gym members from the check-in records on January 9th, 2018
query_gym_members = """
SELECT *
FROM get_fit_now_member
WHERE id IN (SELECT membership_id
             FROM get_fit_now_check_in
             WHERE check_in_date = '20180109'
               AND (check_in_time BETWEEN '1600' AND '1700'
                    OR check_out_time BETWEEN '1600' AND '1700') 
               AND membership_id != 90081);
"""
gym_members = pd.read_sql_query(query_gym_members, conn)
gym_members

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold
1,48Z7A,28819,Joe Germuska,20160305,gold


### Murderer Data

In [56]:

# Enhanced query to gather specific information about the suspect
query_combined_suspects = """
SELECT 
    p.id as person_id, p.name, p.license_id, p.address_number, p.address_street_name, p.ssn,
    d.id as drivers_license_id, d.age, d.height, d.eye_color, d.hair_color, d.gender, d.plate_number,
    i.transcript as interview_transcript,
    inc.annual_income
FROM 
    person p
JOIN 
    drivers_license d ON p.license_id = d.id
JOIN 
    get_fit_now_member m ON p.id = m.person_id
JOIN 
    get_fit_now_check_in c ON m.id = c.membership_id
LEFT JOIN 
    interview i ON p.id = i.person_id
LEFT JOIN 
    income inc ON p.ssn = inc.ssn
WHERE 
    d.gender = 'male'
  AND m.id LIKE '48Z%'
  AND m.membership_status = 'gold'
  AND d.plate_number LIKE '%H42W%'
  AND c.check_in_date = '20180109'
  AND (c.check_in_time BETWEEN '1600' AND '1700'
       OR c.check_out_time BETWEEN '1600' AND '1700')
  AND c.membership_id != 90081;
"""
final_suspect = pd.read_sql_query(query_combined_suspects, conn)
final_suspect

,person_id,name,license_id,address_number,address_street_name,ssn,drivers_license_id,age,height,eye_color,hair_color,gender,plate_number,interview_transcript,annual_income
0,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,423327,30,70,brown,brown,male,0H42W2,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n",10500


In [63]:
# Query to get Facebook event check-in data for the suspect
suspect_id = final_suspect['person_id'][0]
query_facebook_checkins = f"""
SELECT 
    f.person_id, f.event_name, f.date as event_date
FROM 
    facebook_event_checkin f
WHERE 
    f.person_id = {suspect_id};
"""
facebook_checkins = pd.read_sql_query(query_facebook_checkins, conn)
facebook_checkins

,person_id,event_name,event_date
0,67318,The Funky Grooves Tour,20180115
1,67318,SQL Symphony Concert,20171206


### Suspect Woman

In [139]:
# Query to find the suspect based on the provided criteria
query7 = """
SELECT 
    p.id as person_id, 
    p.name, 
    p.license_id, 
    p.address_number, 
    p.address_street_name, 
    p.ssn,
    d.id as drivers_license_id, 
    d.age, 
    d.height, 
    d.eye_color, 
    d.hair_color, 
    d.gender, 
    d.plate_number,
    d.car_model,
    inc.annual_income,
    COUNT(f.person_id) AS total_visit
FROM 
    facebook_event_checkin AS f
JOIN 
    person p ON f.person_id = p.id
JOIN 
    drivers_license d ON p.license_id = d.id
LEFT JOIN 
    income inc ON p.ssn = inc.ssn
WHERE 
    d.gender = 'female' 
    AND d.hair_color = 'red' 
    AND d.car_model = 'Model S'
    AND d.height BETWEEN 65 AND 67
    AND f.event_name = 'SQL Symphony Concert'
    AND f.date LIKE '201712%'
GROUP BY 
    p.id
HAVING total_visit = 3;
"""

suspect = pd.read_sql_query(query7, conn)
suspect

,person_id,name,license_id,address_number,address_street_name,ssn,drivers_license_id,age,height,eye_color,hair_color,gender,plate_number,car_model,annual_income,total_visit
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,202298,68,66,green,red,female,500123,Model S,310000,3


In [136]:
# Query to retrieve all Facebook events for the identified suspect
query_facebook_events = f"""
SELECT *
FROM facebook_event_checkin 
WHERE person_id = {suspect["person_id"][0]};
"""

facebook_events = pd.read_sql_query(query_facebook_events, conn)  
facebook_events

,person_id,event_id,event_name,date
0,99716,1143,SQL Symphony Concert,20171206
1,99716,1143,SQL Symphony Concert,20171212
2,99716,1143,SQL Symphony Concert,20171229


In [138]:
# Query to retrieve the interview transcript for the identified suspect

query_interview = f"""
SELECT *
FROM interview 
WHERE person_id = {suspect["person_id"][0]};
"""

interview = pd.read_sql_query(query_interview, conn)
interview

,person_id,transcript
